# 1. 建立與設定 Raspberry Pi Docker、Azure IoT Hub、Azure IoT Edge、Azure Container Registry

In [ ]:
# 1.1 更新套件
!sudo apt-get update

In [ ]:
# 1.2 安裝 Docker
!sudo apt install -y docker.io

In [ ]:
# 1.3 安裝 Azure CLI
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

In [ ]:
# 1.4 登入 Azure
!az login

In [ ]:
# 1.5 安裝 azure-cli-iot-ext
!az extension add --name azure-cli-iot-ext

In [ ]:
# 1.6 相關資訊設定
# 設定資源群組所在位置
resource_group_location="westus"

# 設定資源群組名稱
date="$(date -d '8hour' +'%Y%m%d')"
resource_group="test"+date

# 設定 IoT Hub 名稱
tmp=!cat /sys/class/net/eth0/address | sed -e "s/://g"
hub_name="for-rpi-"+date+tmp[0]

# 設定 Device ID
device_id="rpi-edge-no-1"

# 設定 Container Registry 名稱
container_registry_name="rpiregistry"+date+tmp[0]
acr_url=container_registry_name+".azurecr.io"

# 設定 module01、module02、module03、module04 與 module05 名稱
module01_name="module01"
module02_name="module02"
module03_name="module03"
module04_name="module04"
module05_name="module05"

# 設定 module01、module02、module03、module04 與 module05 版號
module01_version="0.1"
module02_version="0.1"
module03_version="0.1"
module04_version="0.1"
module05_version="0.1"

In [ ]:
# 1.7 新建資源群組
!az group create -l $resource_group_location -n $resource_group

In [ ]:
# 1.8 建立 IoT Hub
!az iot hub create --resource-group $resource_group --name $hub_name --sku S1

In [ ]:
# 1.9 建立 IoT Edge
!az iot hub device-identity create --hub-name $hub_name --device-id $device_id --edge-enabled

In [ ]:
# 1.10 取得 Azure IoT Edge 的 Connection String
!az iot hub device-identity show-connection-string --resource-group $resource_group --device-id $device_id --hub-name $hub_name

### 在 Raspberry Pi 安裝 Azure IoT Edge
* Ref [安裝 Azure IoT Edge 於 Raspberry Pi 4](https://learningsky.io/install-azure-iot-edge-on-raspberry-pi-4/)

### 在 Raspberry Pi 的終端機輸入下方指令

* 開啟 Raspberry Pi 中的 `config.yaml` 設定檔以修改 `connection string`

  * sudo nano /etc/iotedge/config.yaml
    ```
     provisioning:
     source: "manual"
     device_connection_string: "<ADD DEVICE CONNECTION STRING HERE>"
    ``` 
* 存檔與離開
  * CTRL + X, Y, Enter

* 重新啟動 IoT Edge
  * sudo systemctl restart iotedge    

In [ ]:
# 1.11 建立 Azure Container Registry
!az acr create --resource-group $resource_group --name $container_registry_name --sku Basic

In [ ]:
# 1.12 啟用 Azure Container Registry 的 Admin user
!az acr update --name $container_registry_name --admin-enabled true

In [ ]:
# 1.13 取得 Azure Container Registry 的 name 與 password
registry_name = !az acr list --resource-group $resource_group | grep -m 1 -o '"name": "[^"]*' | grep -o '[^"]*$'
username = !az acr credential show --name $container_registry_name | grep -o '"username": "[^"]*' | grep -o '[^"]*$'
password = !az acr credential show --name $container_registry_name | grep -m 1 -o '"value": "[^"]*' | grep -o '[^"]*$'

# 2. Case1 - 設定與部署 Module01 API 程式

In [ ]:
# 2.1 替換 deployment-1-temp.json 中的文字並存檔成 deployment-1.json
file = open('./deployment-1-temp.json')
contents = file.read()
contents = contents.replace('__ACR_REGISTRY_NAME', registry_name[0])
contents = contents.replace('__ACR_REGISTRY_USER_NAME', username[0])
contents = contents.replace('__ACR_REGISTRY_PASSWORD', password[0])
contents = contents.replace('__MODULE01_NAME', module01_name)
contents = contents.replace('__MODULE01_VERSION', module01_version)

with open('./deployment-1.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# 2.2 在 x86 系統中能 Build ARM 的 Images
!sudo apt-get update && sudo apt-get install -y --no-install-recommends qemu-user-static binfmt-support

In [ ]:
# 2.3 建立 Module01 的 Docker Image
module01_version="0.1"
module01_acr_url=container_registry_name+".azurecr.io/"+module01_name+":"+module01_version
!sudo docker build --rm -f ./Python/Case1/module01/Dockerfile01-1 -t $module01_acr_url ./

In [ ]:
# 2.4 替換 deployment-1-temp.json 中的文字並存檔成 deployment-1.json
file = open('./acr_password-temp.txt')
contents = file.read()
contents = contents.replace('__ACR_REGISTRY_PASSWORD', password[0])

with open('./acr_password.txt', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# 2.5 Azure Container Registry Login
!cat ./acr_password.txt | sudo docker login $acr_url --username $container_registry_name --password-stdin

In [ ]:
# 2.6 將 Docker Image Push 到 Azure Container Registry
!sudo docker push $module01_acr_url

In [ ]:
# 2.7 設定 Azure IoT Hub 中 Edge 的 module
!az iot edge set-modules --resource-group $resource_group --device-id $device_id --hub-name $hub_name --content ./deployment-1.json

### 在 Raspberry Pi 中查看部署狀況
* 指令
  * sudo iotedge list
* 結果
  ![-----2019-12-23---2.35.35](https://s3.amazonaws.com/learningsky/2019/12/-----2019-12-23---2.35.35.png)
  
### 透過瀏覽器測試部署狀況
* 開啟瀏覽器輸入下方網址
  * http://Raspberry-Pi-IP:8080
* 結果
  ```
  Flask Dockerized on Azure IoT Edge ( Version: 0.1 )
  ```

# 3. Case2 - 設定與部署 Module01 傳訊息給 Module02 程式

In [ ]:
# 3.1 建立 Module01-2 的 Docker Image
module01_version="0.2"
module01_acr_url=container_registry_name+".azurecr.io/"+module01_name+":"+module01_version
!sudo docker build --rm -f ./Python/Case2/module01/Dockerfile01-2 -t $module01_acr_url ./

In [ ]:
# 3.2 將 Docker Image Push 到 Azure Container Registry
!sudo docker push $module01_acr_url

In [ ]:
# 3.3 建立 Module02 的 Docker Image
module02_version="0.1"
module02_acr_url=container_registry_name+".azurecr.io/"+module02_name+":"+module02_version
!sudo docker build --rm -f ./Python/Case2/module02/Dockerfile02 -t $module02_acr_url ./

In [ ]:
# 3.4 將 module02 的 Docker Image Push 到 Azure Container Registry
!sudo docker push $module02_acr_url

In [ ]:
# 3.5 替換 deployment-2-temp.json 中的文字並存檔成 deployment-2.json
file = open('./deployment-2-temp.json')
contents = file.read()
contents = contents.replace('__ACR_REGISTRY_NAME', registry_name[0])
contents = contents.replace('__ACR_REGISTRY_USER_NAME', username[0])
contents = contents.replace('__ACR_REGISTRY_PASSWORD', password[0])
contents = contents.replace('__MODULE01_NAME', module01_name)
contents = contents.replace('__MODULE01_VERSION', module01_version)
contents = contents.replace('__MODULE02_NAME', module02_name)
contents = contents.replace('__MODULE02_VERSION', module02_version)

with open('./deployment-2.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# 3.6 設定 Azure IoT Hub 中 Edge 的 module
!az iot edge set-modules --resource-group $resource_group --device-id $device_id --hub-name $hub_name --content ./deployment-2.json

### 在 Raspberry Pi 中查看部署狀況
* 指令
  * sudo iotedge list
* 結果
  ![-----2019-12-23---3.50.11](https://s3.amazonaws.com/learningsky/2019/12/-----2019-12-23---3.50.11.png)
  
### 測試部署狀況
* 瀏覽器端
  * 開啟瀏覽器輸入下方網址
    * http://Raspberry-Pi-IP:8080/message/hello_world
  * 瀏覽器顯示結果
    ```
    Message - hello_world!
    ```
* Raspberry Pi 端
  * 請輸入下方指令
    ```
    sudo iotedge logs module02
    ```
  * 顯示結果
    ```
    Received - hello_world!
    ```

# 4. Case3 - 設定與部署 Module03 傳訊息給 Azure IoT Hub 程式 <br>  ( send_message_to_output )

In [ ]:
# 4.1 建立 Module03 的 Docker Image
module03_acr_url=container_registry_name+".azurecr.io/"+module03_name+":"+module03_version
!sudo docker build --rm -f ./Python/Case3/module03/Dockerfile03 -t $module03_acr_url ./

In [ ]:
# 4.2 將 module03 的 Docker Image Push 到 Azure Container Registry
!sudo docker push $module03_acr_url

In [ ]:
# 4.3 替換 deployment-3-temp.json 中的文字並存檔成 deployment-3.json
file = open('./deployment-3-temp.json')
contents = file.read()
contents = contents.replace('__ACR_REGISTRY_NAME', registry_name[0])
contents = contents.replace('__ACR_REGISTRY_USER_NAME', username[0])
contents = contents.replace('__ACR_REGISTRY_PASSWORD', password[0])
contents = contents.replace('__MODULE01_NAME', module01_name)
contents = contents.replace('__MODULE01_VERSION', module01_version)
contents = contents.replace('__MODULE02_NAME', module02_name)
contents = contents.replace('__MODULE02_VERSION', module02_version)
contents = contents.replace('__MODULE03_NAME', module03_name)
contents = contents.replace('__MODULE03_VERSION', module03_version)

with open('./deployment-3.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# 4.4 設定 Azure IoT Hub 中 Edge 的 module
!az iot edge set-modules --resource-group $resource_group --device-id $device_id --hub-name $hub_name --content ./deployment-3.json

In [ ]:
# 4.5 從 Azure IoT Hub 查看 Device to Cloud (D2C) 的訊息
!az iot hub monitor-events --y --resource-group $resource_group --hub-name $hub_name --device-id $device_id

# 5. Case4 - 設定與部署 Module04 傳訊息給 Azure IoT Hub 程式 <br>  ( send_message )

In [ ]:
# 5.1 取得 Azure IoT Edge 的 Connection String
connection_string = !az iot hub device-identity show-connection-string --resource-group $resource_group --device-id $device_id --hub-name $hub_name | grep -o '"connectionString": "[^"]*' | grep -o '[^"]*$'
connection_string = "CONNECTION_STRING=" + connection_string[0]
with open(".env", 'wt', encoding='utf-8') as output_file:
    output_file.write(connection_string)

In [ ]:
# 5.2 建立 Module04 的 Docker Image
module04_version="0.19"
module04_acr_url=container_registry_name+".azurecr.io/"+module04_name+":"+module04_version
!sudo docker build --rm -f ./Python/Case4/module04/Dockerfile04 -t $module04_acr_url ./

In [ ]:
# 5.3 將 module04 的 Docker Image Push 到 Azure Container Registry
!sudo docker push $module04_acr_url

In [ ]:
# 5.4 替換 deployment-4-temp.json 中的文字並存檔成 deployment-4.json
file = open('./deployment-4-temp.json')
contents = file.read()
contents = contents.replace('__ACR_REGISTRY_NAME', registry_name[0])
contents = contents.replace('__ACR_REGISTRY_USER_NAME', username[0])
contents = contents.replace('__ACR_REGISTRY_PASSWORD', password[0])
contents = contents.replace('__MODULE04_NAME', module04_name)
contents = contents.replace('__MODULE04_VERSION', module04_version)

with open('./deployment-4.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# 5.5 設定 Azure IoT Hub 中 Edge 的 module
!az iot edge set-modules --resource-group $resource_group --device-id $device_id --hub-name $hub_name --content ./deployment-4.json

In [ ]:
# 5.6 從 Azure IoT Hub 查看 Device to Cloud (D2C) 的訊息
!az iot hub monitor-events --y --resource-group $resource_group --hub-name $hub_name --device-id $device_id

# 6. Case5 - 設定與部署 Module05 接收來自 Azure IoT Hub 的訊息 <br>  ( receive_message )

In [ ]:
# 6.1 取得 Azure IoT Edge 的 Connection String
connection_string = !az iot hub device-identity show-connection-string --resource-group $resource_group --device-id $device_id --hub-name $hub_name | grep -o '"connectionString": "[^"]*' | grep -o '[^"]*$'
connection_string = "CONNECTION_STRING=" + connection_string[0]
with open(".env", 'wt', encoding='utf-8') as output_file:
    output_file.write(connection_string)

In [ ]:
# 6.2 建立 Module05 的 Docker Image
module05_version="0.3"
module05_acr_url=container_registry_name+".azurecr.io/"+module05_name+":"+module05_version
!sudo docker build --rm -f ./Python/Case5/module05/Dockerfile05 -t $module05_acr_url ./

In [ ]:
# 6.3 將 module05 的 Docker Image Push 到 Azure Container Registry
!sudo docker push $module05_acr_url

In [ ]:
# 6.4 替換 deployment-5-temp.json 中的文字並存檔成 deployment-5.json
file = open('./deployment-5-temp.json')
contents = file.read()
contents = contents.replace('__ACR_REGISTRY_NAME', registry_name[0])
contents = contents.replace('__ACR_REGISTRY_USER_NAME', username[0])
contents = contents.replace('__ACR_REGISTRY_PASSWORD', password[0])
contents = contents.replace('__MODULE05_NAME', module05_name)
contents = contents.replace('__MODULE05_VERSION', module05_version)

with open('./deployment-5.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# 6.5 設定 Azure IoT Hub 中 Edge 的 module
!az iot edge set-modules --resource-group $resource_group --device-id $device_id --hub-name $hub_name --content ./deployment-5.json

In [ ]:
# 6.6 從 Azure IoT Hub 傳送訊息到 Device - Cloud to Device (C2D)
message="'Hello World'"
!az iot device c2d-message send --resource-group $resource_group --hub-name $hub_name --device-id $device_id  --data $message

### 測試部署狀況
* Raspberry Pi 端
  * 請輸入下方指令
    ```
    sudo iotedge logs module05
    ```
  * 顯示結果
    ```
    Received - hello_world!
    ```

# 7. 移除相關設定

In [ ]:
# 7.1 移除 deployment.json & acr_password.txt
!rm deployment-1.json deployment-2.json deployment-3.json acr_password.txt

In [ ]:
# 7.2 移除 Azure IoT Hub & Azure Container Registry & Resource Group
!az acr delete --name $container_registry_name --resource-group $resource_group
!az iot hub delete --name $hub_name --resource-group $resource_group
!az group delete --yes --name $resource_group